__tools for test data and otherwise accessing canvas as a student__

In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
# %matplotlib inline
# from matplotlib import pyplot as plt
# import seaborn as sns
# sns.set(style="whitegrid")

import pandas as pd

from CanvasHacks import environment
from CanvasHacks.RequestTools import *
from CanvasHacks.UrlTools import *
from CanvasHacks.Configuration import InteractiveConfiguration
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.Repositories.DataManagement import DataStore

from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.UploadGradeTools import make_upload_button

# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator
from CanvasHacks.JournalsFileTools import get_journal_folders, make_folder_list, calculate_journal_counts
from CanvasHacks.JournalsFileTools import journal_folder_name, create_folder


from CanvasHacks.Models.student import Student

# Import the Canvas class
import canvasapi
from canvasapi import Canvas
from canvasapi.quiz import QuizReport, Quiz
# from canvasapi.requester import Requester
# from canvasapi.conversation import Conversation

# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)
# requester = Requester(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)

import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))

from CanvasHacks.PeerReviewed.Definitions import Review, InitialWork, MetaReview, Unit


from CanvasHacks.Repositories.quizzes import QuizRepository

# Widgets
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments
from CanvasHacks.Widgets.AssignmentSelection import make_discussion_chooser
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser


# Testing
from faker import Faker
fake = Faker()
from CanvasHacks.Models.users import StudentUser
from tests.factories.PeerReviewedFactories import discussion_entry_factory
from tests.factories.SKAAFactories import populate_quiz_w_text_answers
from tests.factories.SKAAFactories import populate_discussion, populate_assignment, answer_survey

STUDENT_TEST_ACCT_FP = "{}/private/test-accounts.xlsx".format(environment.CONFIG.proj_base)


In [ ]:


main_quiz = 165098
review_quiz = 165820
meta_review = 165821

url = 'https://canvas.csun.edu/api/v1/courses/85210/assignments/{}/submissions'

# last semester data
prev_course_id = 62657
prev_quiz = 151633

# QUIZ_ID = prev_quiz

TEST_STUDENT_ID = 168439
SEMESTER_NAME = 'S20'
topic_id = 737847


adam_id = 6417

# Test data
test_course_id = 85210

COURSE_ID = test_course_id
# COURSE_ID = prev_course_id
SEMESTER_NAME = 'S20'
UNIT_NUMBER = 1

In [ ]:
STUDENT_TEST_ACCT_FP = "{}/private/test-accounts.xlsx".format(environment.CONFIG.proj_base)
student_users_df = pd.read_excel(STUDENT_TEST_ACCT_FP)
tokens = student_users_df.token.dropna().tolist()


In [ ]:
make_test_selector()
make_unit_chooser()

In [ ]:
# Load definitions of unit components
# course = canvas.get_course(COURSE_ID)
# unit = Unit(course, UNIT_NUMBER)

canvas = environment.CONFIG.canvas
course = environment.CONFIG.course
unit = environment.CONFIG.unit


# Populate survey

## Peer review

In [ ]:
questions = course.get_quiz(unit.review.quiz_id).get_questions()
essay_questions = [q for q in questions if q.question_type == 'essay_question']
multiple_choice = [q for q in questions if q.question_type == 'multiple_choice_question']

answers = []
for t in tokens:
    a = answer_survey(t, COURSE_ID, unit.review.quiz_id, essay_questions, multiple_choice)
    answers.append(a)
answers

## Metareview

In [ ]:
questions = course.get_quiz(unit.metareview.quiz_id).get_questions()
essay_questions = [q for q in questions if q.question_type == 'essay_question']
multiple_choice = [q for q in questions if q.question_type == 'multiple_choice_question']

answers = []
for t in tokens:
    a = answer_survey(t, COURSE_ID, unit.metareview.quiz_id, essay_questions, multiple_choice)
    answers.append(a)
answers


# Populate quiz

The process for creating dummy data for a quiz is as follows:

- As instructor, get list of questions on the quiz

- As student, create submission
s = s1.assignment.create_submission()

- submit answers

.answer_submission_questions(quiz_questions=[{
    "id": "3134771",
    "answer": "Hello World! d"
  }])
  
  - mark submission complete

In [ ]:
populate_quiz_w_text_answers(COURSE_ID, quiz_id, tokens)

# Populate assignments

In [ ]:
assign_id = 640461

In [ ]:
populate_assignment(tokens, assign_id, COURSE_ID)

# Populate discussion forum

In [ ]:
make_discussion_chooser(course)

In [ ]:
depth =2
for topic_id, name in environment.CONFIG.discussions:
    populate_discussion(tokens, topic_id, depth, COURSE_ID)

# Attic

In [ ]:
# def get_question_ids(course, quiz_id):
#     """Returns list of ids for questions comprising quiz"""
#     qids = [q.id for q in course.get_quiz(quiz_id).get_questions()]
#     print("{} question ids retrieved for quiz {}".format(len(qids), quiz_id))
#     return qids


# def answer_quiz_for_student(student_token, course_id, quiz_id, question_ids):
#     student = StudentUser(student_token, course_id, quiz_id=quiz_id)
#     quiz = student.course.get_quiz(quiz_id)
    
#     # Create a submission object
#     try:
#         submission = quiz.create_submission()
#     except canvasapi.exceptions.Conflict:
#         # a submission was already created
#         # so get that one
#         submission = [s for s in quiz.get_submissions()][0]

        
#     # make answers
#     answers = []
#     for qid in question_ids:
#         answers.append({
#             'id' : qid,
#             'answer': fake.paragraph()
#         })
    
#     # submit answers
#     question_subs = submission.answer_submission_questions(quiz_questions=answers)
    
#     # Complete the quiz
#     submission.complete()
#     print("Answers created for user {}".format(submission.user_id))

#     # return quizSubmissionQuestions for use in testing
#     # answer text will be stored on answer attribute
#     return question_subs


# def populate_quiz_w_text_answers(course, quiz_id, student_tokens):
#     """
#     Answers all text entry questions for a quiz for all students
    
#     Returns a list of QuizSubmissionQuestions which contains the answer text
#     in answer attribute"""
#     answers = []
#     question_ids = get_question_ids(course, quiz_id)
#     for t in student_tokens:
#         r = answer_quiz_for_student(t, course.id, quiz_id, question_ids)
#         answers.append(r)
#     return answers



In [ ]:
# def populate_assignment(tokens, assignment_id, course_id=COURSE_ID):
#     """Uploads a paragraph of response text for each token provided
#     in tokens. 
#     Returns a list of the submission objects returned"""
#     submissions = []
    
#     for t in tokens:
#         msg = fake.paragraph()
#         sub = {
#             'submission_type': 'online_text_entry',
#             'body': msg,
#         }
#         s1 = StudentUser(t, course_id, assignment_id=assignment_id)
#         result = s1.assignment.submit(sub)
#         submissions.append(result)
#     return submissions

In [ ]:
# def populate_discussion(tokens, topic_id, deep, course_id=COURSE_ID):
#     posts = []
#     student_users = []
#     for t in tokens:
#         student_users.append(StudentUser(t, COURSE_ID, topic_id=topic_id))

#     for su in student_users:
#         msg = fake.paragraph()
#         e = su.post_entry(msg)
#         posts.append(e)
#     for i in range(0, deep):
#         for s1 in student_users:
#             for entry in s1.discussion_entries:
#                 # every other student comments on each entry
#                 for s2 in student_users:
#                     msg = fake.paragraph()
#                     e = s2.discussion.get_entries([entry.id])
#                     r = e[0].post_reply(message=msg)
#                     posts.append(r)

In [ ]:
# def answer_quiz_for_student(student_token, course_id, quiz_id, question_ids):
#     student = StudentUser(student_token, course_id, quiz_id=quiz_id)
#     quiz = student.course.get_quiz(quiz_id)
    
#     # Create a submission object
#     try:
#         submission = quiz.create_submission()
#     except canvasapi.exceptions.Conflict:
#         # a submission was already created
#         # so get that one
#         submission = [s for s in quiz.get_submissions()][0]

        
#     # make answers
#     answers = []
#     for qid in question_ids:
#         answers.append({
#             'id' : qid,
#             'answer': fake.paragraph()
#         })
    
#     # submit answers
#     question_subs = submission.answer_submission_questions(quiz_questions=answers)
    
#     # Complete the quiz
#     submission.complete()
#     print("Answers created for user {}".format(submission.user_id))

#     # return quizSubmissionQuestions for use in testing
#     # answer text will be stored on answer attribute
#     return question_subs


In [ ]:
msg = fake.paragraph()
entry = s_discussion.post_entry(message=msg)


In [ ]:
entry.post_reply(message=fake.paragraph())

In [ ]:
def make_course(token, course_id=COURSE_ID):
    # Student account
    canvas = Canvas(environment.CONFIG.canvas_url_base, token)
    course = canvas.get_course(course_id)
    return course